<a href="https://colab.research.google.com/github/nicolemichaud03/Chatbot_Preference_Predictor/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Competition Submission: "LMSYS - Chatbot Arena Human Preference Predictions" (Notebook)

## Intro

In [9]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#upload the kaggle.json file to load kaggle data
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nicolemichaud","key":"e7fb46517234dcc36218486df964d630"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
!pip install -q kaggle

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
#download the data from kaggle
!kaggle competitions download -c lmsys-chatbot-arena

 81% 46.0M/57.0M [00:00<00:00, 87.6MB/s]
100% 57.0M/57.0M [00:00<00:00, 80.0MB/s]


In [7]:
#unzip data from kaggle
import zipfile

# Define the path to your zip file
file_path = '/content/lmsys-chatbot-arena.zip'
# Unzip the file to a specific destination
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/Colab Notebooks')


In [11]:
#read in the specific datasets to be used:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')

## Data Exploration

In [12]:
train_data.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [13]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57477 entries, 0 to 57476
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              57477 non-null  int64 
 1   model_a         57477 non-null  object
 2   model_b         57477 non-null  object
 3   prompt          57477 non-null  object
 4   response_a      57477 non-null  object
 5   response_b      57477 non-null  object
 6   winner_model_a  57477 non-null  int64 
 7   winner_model_b  57477 non-null  int64 
 8   winner_tie      57477 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 3.9+ MB


In [14]:
train_data.describe()

,id,winner_model_a,winner_model_b,winner_tie
count,5.747700e+04,57477.000000,57477.000000,57477.000000
mean,2.142564e+09,0.349079,0.341911,0.309011
std,1.238327e+09,0.476683,0.474354,0.462090
min,3.019200e+04,0.000000,0.000000,0.000000
25%,1.071821e+09,0.000000,0.000000,0.000000
50%,2.133658e+09,0.000000,0.000000,0.000000
75%,3.211645e+09,1.000000,1.000000,1.000000
max,4.294947e+09,1.000000,1.000000,1.000000


In [15]:
train_data['model_a'].unique()

array(['gpt-4-1106-preview', 'koala-13b', 'gpt-3.5-turbo-0613',
       'llama-2-13b-chat', 'vicuna-13b', 'mixtral-8x7b-instruct-v0.1',
       'gemini-pro', 'gpt-4-0314', 'vicuna-7b', 'chatglm3-6b',
       'pplx-70b-online', 'mpt-30b-chat', 'llama2-70b-steerlm-chat',
       'claude-1', 'claude-2.1', 'chatglm-6b', 'claude-instant-1',
       'dolly-v2-12b', 'claude-2.0', 'deepseek-llm-67b-chat',
       'openchat-3.5', 'starling-lm-7b-alpha', 'gpt-4-0125-preview',
       'llama-2-7b-chat', 'gpt-4-0613', 'wizardlm-70b',
       'stablelm-tuned-alpha-7b', 'vicuna-33b', 'chatglm2-6b',
       'dolphin-2.2.1-mistral-7b', 'llama-2-70b-chat', 'llama-13b',
       'palm-2', 'wizardlm-13b', 'codellama-34b-instruct',
       'gemini-pro-dev-api', 'gpt-3.5-turbo-0314', 'gpt-3.5-turbo-1106',
       'yi-34b-chat', 'oasst-pythia-12b', 'qwen-14b-chat', 'alpaca-13b',
       'qwen1.5-72b-chat', 'gpt-3.5-turbo-0125', 'pplx-7b-online',
       'qwen1.5-4b-chat', 'fastchat-t5-3b', 'solar-10.7b-instruct-v1.0',
   

In [16]:
train_data['model_b'].unique()

array(['gpt-4-0613', 'mistral-medium', 'mistral-7b-instruct',
       'gpt-3.5-turbo-0314', 'gpt-4-0314', 'vicuna-13b', 'claude-2.0',
       'gpt-3.5-turbo-0613', 'guanaco-33b', 'openchat-3.5',
       'gpt-3.5-turbo-1106', 'gpt4all-13b-snoozy', 'koala-13b',
       'gemini-pro-dev-api', 'wizardlm-70b', 'claude-instant-1',
       'alpaca-13b', 'gpt-4-0125-preview', 'wizardlm-13b',
       'gpt-4-1106-preview', 'claude-2.1', 'chatglm2-6b',
       'tulu-2-dpo-70b', 'claude-1', 'zephyr-7b-beta',
       'starling-lm-7b-alpha', 'oasst-pythia-12b', 'llama-2-13b-chat',
       'codellama-34b-instruct', 'yi-34b-chat', 'mpt-7b-chat',
       'vicuna-33b', 'deepseek-llm-67b-chat', 'llama-2-70b-chat',
       'fastchat-t5-3b', 'gpt-3.5-turbo-0125', 'chatglm-6b',
       'openhermes-2.5-mistral-7b', 'llama-2-7b-chat', 'pplx-70b-online',
       'mixtral-8x7b-instruct-v0.1', 'RWKV-4-Raven-14B',
       'solar-10.7b-instruct-v1.0', 'chatglm3-6b', 'dolly-v2-12b',
       'falcon-180b-chat', 'palm-2', 'qwen1.5-7

## Modeling